I will be conducting an independent samples t-test, looking at the effect of temperature (positive or negative) on weekly sales. This dataset contains data released by Walmart and it can be found at https://www.kaggle.com/datasets/yasserh/walmart-dataset

In [32]:
library(readr)
walmart <- read_csv("Walmart.csv")
walmart <- as.data.frame(walmart)
dim(walmart)
head(walmart)

Rows: 6435 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Date
dbl (7): Store, Weekly_Sales, Holiday_Flag, Temperature, Fuel_Price, CPI, Un...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 6435    8

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,05-02-2010,1643691,0,42.31,2.572,211.0964,8.106
2,1,12-02-2010,1641957,1,38.51,2.548,211.2422,8.106
3,1,19-02-2010,1611968,0,39.93,2.514,211.2891,8.106
4,1,26-02-2010,1409728,0,46.63,2.561,211.3196,8.106
5,1,05-03-2010,1554807,0,46.50,2.625,211.3501,8.106
6,1,12-03-2010,1439542,0,57.79,2.667,211.3806,8.106


In [33]:
dim(walmart)

[1] 6435    8

In [34]:
any(is.na(walmart)) # to check if there are any missing values

[1] FALSE

In [35]:
walmart$Temperature <- (walmart$Temperature - 32) * 5/9 # convert temperature from fahrenheit to celcius
walmart$temperature_category <- walmart$Temperature
head(walmart)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,temperature_category
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,05-02-2010,1643691,0,5.727778,2.572,211.0964,8.106,5.727778
2,1,12-02-2010,1641957,1,3.616667,2.548,211.2422,8.106,3.616667
3,1,19-02-2010,1611968,0,4.405556,2.514,211.2891,8.106,4.405556
4,1,26-02-2010,1409728,0,8.127778,2.561,211.3196,8.106,8.127778
5,1,05-03-2010,1554807,0,8.055556,2.625,211.3501,8.106,8.055556
6,1,12-03-2010,1439542,0,14.327778,2.667,211.3806,8.106,14.327778


A for loop that replaces each value in the newly created temperature_category column (which is a copy of values in the Temperature column) with the string value of "Positive_Temperature" or  "Negative_Temperature".

In [36]:
for(i in 1:length(walmart$temperature_category)){
  if(walmart$temperature_category[i] >= 0){
    walmart$temperature_category[i] <- "Positive_Temperature"
  } else {
    walmart$temperature_category[i] <- "Negative_Temperature"
  }
}

walmart$temperature_category <- as.factor(walmart$temperature_category)

head(walmart)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,temperature_category
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,05-02-2010,1643691,0,5.727778,2.572,211.0964,8.106,Positive_Temperature
2,1,12-02-2010,1641957,1,3.616667,2.548,211.2422,8.106,Positive_Temperature
3,1,19-02-2010,1611968,0,4.405556,2.514,211.2891,8.106,Positive_Temperature
4,1,26-02-2010,1409728,0,8.127778,2.561,211.3196,8.106,Positive_Temperature
5,1,05-03-2010,1554807,0,8.055556,2.625,211.3501,8.106,Positive_Temperature
6,1,12-03-2010,1439542,0,14.327778,2.667,211.3806,8.106,Positive_Temperature


I will now use the Levene's Test to check for homogeneity/equality of variances and use the the Kolmogorov-Smirnov Test to check if the data is normally distributed. Since the size of walmart$Weekly_Sales is greater than 5000, we will not be to use the Shapiro-Wilk Normality test.

In [26]:
install.packages('car')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [38]:
library('car')
leveneTest(Weekly_Sales ~ temperature_category, walmart)

,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,1,16.53413,4.835241e-05
,6433,NA,NA


In [39]:
# Since the size of walmart$Weekly_Sales is greater than 5000, we will not be to use the Shapiro-Wilk Normality test. Therefore, the Kolmogorov-Smirnov Test will be used instead
ks.test(walmart$Weekly_Sales, 'pnorm')


	Asymptotic one-sample Kolmogorov-Smirnov test

data:  walmart$Weekly_Sales
D = 1, p-value < 2.2e-16
alternative hypothesis: two-sided


The samples are clearly not normally distributed and neither is there equality/homogeneity of variance so use the Mann–Whitney U Test/Wilcoxon Rank-Sum Test instead.


In [40]:
wilcox.test(Weekly_Sales ~ temperature_category, data = walmart, alternative = "two.sided")


	Wilcoxon rank sum test with continuity correction

data:  Weekly_Sales by temperature_category
W = 1530379, p-value = 0.8825
alternative hypothesis: true location shift is not equal to 0


The results suggest that there is no signficant difference between sales on negative temperature days and sales on positive temperature days. Since positive temperature days are temporally clustered (they usually  occur during the spring, summer, fall, etc.) and so are negative temperature days (they usually occur during the winter), it is fair to consider weekly sales to be a general measure of sales for a particular temperature category.

  Below is what the results would have looked like if I had used a t-test instead.


In [43]:
t.test(walmart$Weekly_Sales ~ walmart$temperature_category, alternative = "two.sided")


	Welch Two Sample t-test

data:  walmart$Weekly_Sales by walmart$temperature_category
t = 0.16537, df = 606.46, p-value = 0.8687
alternative hypothesis: true difference in means between group Negative_Temperature and group Positive_Temperature is not equal to 0
95 percent confidence interval:
 -46729.09  55322.18
sample estimates:
mean in group Negative_Temperature mean in group Positive_Temperature 
                           1050918                            1046621 


Even after using a statistically more powerful test, there's still no significant effect.